In [1]:
import time
from datetime import timedelta
import html

import pandas as pd
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Read csv files Java Answers

In [2]:
df = dd.read_csv('checkstylexmlreports_csv/checkstyleReport*.csv', engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)


## Start a Dask cluster using SLURM jobs as workers

In [3]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39456 instead
  warnings.warn(


Spawn between 20 to 100 workers and connect a client to be able use them.

In [4]:
#cluster.scale(n=20) # ask for 20 jobs or workers
# This also works with adaptive clusters. This automatically launches and kill workers based on load.
# we tell our cluster to autoscale between 10 and 20 workers depending on the load
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
#cluster.adapt(maximum_jobs=20)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.154:43462 Dashboard: http://192.168.94.154:39456/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
df

,name,line,severity,message,source
npartitions=115,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [7]:
checkstyle_codeviolations_init_len = df.shape[0].compute()
print('Initial number of volations {}'.format(checkstyle_codeviolations_init_len))

Initial number of volations 8220118


In [9]:
# drop empty columns
df = df[~df['name'].isna()] # Drop rows that have NaN in the Code column

In [10]:
checkstyle_codeviolations_nonan_len =df.shape[0].compute()
print('Number of violations after removing the nans in the class column {}'.format(checkstyle_codeviolations_nonan_len))

Number of violations after removing the nans in the class column 8220118


#### Get the unique classes dataframe

In [12]:
# get the unique classes
unique_classes_df = df['name'].drop_duplicates().compute()

In [13]:
# This also counts the unique classes
unique_classes_df.shape[0]

676575